In [8]:
from dotenv import load_dotenv
import os
import pymysql as pm

# load .env
load_dotenv()

conn = pm.connect(
    host=os.environ.get('SQLHOST'),
    port=int(os.environ.get('SQLPORT')),
    user=os.environ.get('SQLUSER'),
    password=os.environ.get('SQLPW'),
    charset='utf8'
)

cursor = conn.cursor()
cursor.execute("USE wrtn_char")

create_test = ''' CREATE TABLE `test` (
	`name` VARCHAR(50) NOT NULL,
	`img` BLOB NOT NULL,
    `ext` VARCHAR(5) NOT NULL DEFAULT 'jpeg',
	PRIMARY KEY (`name`),
	UNIQUE KEY (`name`)
);'''
try:
    cursor.execute(create_test)
except:
    print("pass")

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
prefs = {"download.default_directory": os.path.join(os.getcwd(),"img")}
options.add_experimental_option("prefs", prefs)
# 옵션 적용 시
driver = webdriver.Chrome(options=options)

# '캐릭터 둘러보기' 페이지 이동
driver.get("https://wrtn.ai/character")

# 카테고리 선택
cat_num = 3
cat = driver.find_element(By.CSS_SELECTOR, f'.css-1fzkvcn > *:nth-child({cat_num})')
# db에 저장할 카테고리명 추출
db_category = cat.text
# 해당 카테고리 클릭
cat.click()

# #스크롤 가능 오브젝트가 윈도우가 아니라, #character-home-scroll 다.
# scrollable_div = driver.find_element(By.CSS_SELECTOR, "#character-home-scroll")
# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

time.sleep(2)
# 특정 캐릭터 선택
dataIdx= 2
each_selector = f'.virtuoso-grid-item[data-index="{dataIdx}"]'
# 해당 캐릭터 요소를 element에 저장
element = driver.find_element(By.CSS_SELECTOR, each_selector)

# 해당 요소를 text로 파싱한 후, 개행 문자 기준으로 나눠서 캐릭터명, 설명, 작성자를 저장
db_name, db_comment, db_writer = element.text.split('\n')

# 클릭해서 들어가는 요소와, 이미지 요소의 선택자를 별도 작성
clk_selector = each_selector+'>div>div:first-child'
img_selector = clk_selector+'>div:first-child>.character_avatar>img'

# 이미지 요소의 url을 저장
db_img = driver.find_element(By.CSS_SELECTOR, img_selector).get_attribute('src')

In [28]:
db_img_name, db_img_ext = db_img.split('/')[-1].split('.')
print(db_img_name, ',', db_img_ext)

4e5ef0b3-f2a4-4788-82c1-af2b9a3ad271_w600 , webp


In [29]:
driver.get(db_img)

img_dir = os.path.join("./img", db_img_name+'.'+db_img_ext)

In [39]:
from io import BytesIO
import base64
from PIL import Image

buffer = BytesIO()
im = Image.open(img_dir)
im.save(buffer, format=db_img_ext)
img_str = base64.b64encode(buffer.getvalue())
img_str = img_str.decode('UTF-8')
cursor.execute(f"INSERT INTO test (name, img, ext) VALUES ('{db_name}', '{img_str}', '{db_img_ext}')")

1

In [40]:
import pandas as pd

cursor.execute("SELECT * FROM test")
result = cursor.fetchall()

df = pd.DataFrame(result)
print(df)

     0                                                  1     2
0  차영빈  b'UklGRqx8AABXRUJQVlA4IKB8AACwngKdASpYAlgCPm0y...  webp


In [44]:
os.remove(img_dir)

In [37]:
driver.quit()